In [24]:
import pandas as pd
import numpy as np

In [7]:
# numpy.concatenate()
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]

print(np.concatenate([x, y, z]))
print(np.vstack([x, y, z]))

# vstack() : concatenate verically
# hstack() : concatenate horizon

x = [[1, 2], [3, 4]] # 2D array

# axis = 0 (row-based), 1 (column-based)
np.concatenate([x, x], axis=1)

[1 2 3 4 5 6 7 8 9]
[[1 2 3]
 [4 5 6]
 [7 8 9]]


array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

In [10]:
# pandas.concat()
# objs : 연결대상 객체들
# axis : 연결 차원 축
# join : 조인 옵션
# ignore_index : 기존의 인덱스 무시
ser1 = pd.Series(['A', 'B', 'C'], index=[1,2,3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4,5,6])

print(ser1)
print(ser2, '\n')
print(pd.concat([ser1, ser2]))
print(pd.concat([ser2, ser1]))

1    A
2    B
3    C
dtype: object
4    D
5    E
6    F
dtype: object 

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object
4    D
5    E
6    F
1    A
2    B
3    C
dtype: object


In [11]:
# DataFrame 생성 함수 정의
def make_df(cols, ind) :
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

print(make_df('ABC', [1,2]))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2


In [13]:
# concatenate the two dataframe objects
df1 = make_df('AB', [1,2])
df2 = make_df('AB', [3,4])

print(pd.concat([df1, df2]))

df3 = make_df('AB', [0,1])
df4 = make_df('CD', [0,1])

# axis = 0 or 1
print(pd.concat([df3,df4], axis=1)) # 수평방향으로 합치기

    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1


In [18]:
# concate() Features

# Case 1 : 인덱스 중복 허용
x = make_df('AB', [0,1])
y = make_df('AB', [2,3])

y.index = x.index

print(pd.concat([x,y]))

# Case 2 : 인덱스 중복 에러 처리 (verify_integrity=True)
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print('valueError: ', e)

# Case 3 : 인덱스 무시 (ignore_index=True)
print(pd.concat([x,y], ignore_index=True))

# Case 4 : 계층적 인덱싱 지원 : keys = [key]

    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3
valueError:  Indexes have overlapping values: Int64Index([0, 1], dtype='int64')
    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


In [20]:
# concat([ ], join = outer(합집합) or inner(교집합))
x = make_df('ABC', [0,1,2])
y = make_df('BCD', [3,4,5])

print(pd.concat([x, y], join='outer'))
print(pd.concat([x, y], join='inner'))

     A   B   C    D
0   A0  B0  C0  NaN
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4
5  NaN  B5  C5   D5
    B   C
0  B0  C0
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4
5  B5  C5


In [ ]:
# 조인 연산 : join() 인덱스를 기준으로 두 DataFrame을 결합
# 병합 연산 : merge() 데이터셋 조인/병합 관련 다양한 옵션 제공

In [31]:
# 일대일 조인 : 공통 열의 중복 항목 없음

df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                   'group': ['Accounting', 'Enginerring', 'Enginerring', 'HR']})

df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                   'hire_date': [2004, 2008, 2012, 2014]})
print(df1, '\n')
print(df2, '\n')

df3 = pd.merge(df1, df2)
print(df3)

  employee        group
0      Bob   Accounting
1     Jake  Enginerring
2     Lisa  Enginerring
3      Sue           HR 

  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014 

  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Enginerring       2012
2     Lisa  Enginerring       2004
3      Sue           HR       2014


In [36]:
# 다대일 조인 : 두 DataFrame의 키 중에 하나가 중복된 항목

df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                   'supervisor': ['Carly', 'Guido', 'Steve']})

print(df3, '\n')
print(df4, '\n')

print(pd.merge(df3, df4)) # df3 = left, df4 = right

  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Enginerring       2012
2     Lisa  Enginerring       2004
3      Sue           HR       2014 

         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve 

  employee       group  hire_date supervisor
0      Bob  Accounting       2008      Carly
1      Sue          HR       2014      Steve


In [45]:
# 다대다 조인 : 두 DataFrame의 키가 모두 중복된 항목을 포함
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                             'Engineering', 'Engineering',
                             'HR', 'HR'],
                   'skills': ['math', 'spreadsheets', 'coding',
                             'linux', 'spreadsheets', 'organization']})

print(df1, '\n')
print(df5, '\n')

print(pd.merge(df1, df5))

  employee        group  age
0      Bob   Accounting   20
1     Jake  Enginerring   42
2     Lisa  Enginerring   37
3      Sue           HR   25 

         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization 

  employee       group  age        skills
0      Bob  Accounting   20          math
1      Bob  Accounting   20  spreadsheets
2      Sue          HR   25  spreadsheets
3      Sue          HR   25  organization


In [41]:
print(df1, '\n')
print(df2, '\n')

df1['age'] = pd.Series([20, 42, 37, 25])
df2['age'] = pd.Series([37, 20, 42, 35])
print(df1, '\n')
print(df2, '\n')

# on : 병합에 사용할 공통 키
print(pd.merge(df1, df2, on="employee"))

  employee        group  age
0      Bob   Accounting   20
1     Jake  Enginerring   42
2     Lisa  Enginerring   37
3      Sue           HR   25 

  employee  hire_date  age
0     Lisa       2004   37
1      Bob       2008   20
2     Jake       2012   42
3      Sue       2014   35 

  employee        group  age
0      Bob   Accounting   20
1     Jake  Enginerring   42
2     Lisa  Enginerring   37
3      Sue           HR   25 

  employee  hire_date  age
0     Lisa       2004   37
1      Bob       2008   20
2     Jake       2012   42
3      Sue       2014   35 

  employee        group  age_x  hire_date  age_y
0      Bob   Accounting     20       2008     20
1     Jake  Enginerring     42       2012     42
2     Lisa  Enginerring     37       2004     37
3      Sue           HR     25       2014     35


In [46]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                   'salary': [70000, 80000, 120000, 90000]})

print(df1, '\n')
print(df3, '\n')

# left_on, right_on : 공통 키 역할을 하는 두 DataFrame의 열
pd.merge(df1, df3, left_on="employee", right_on="name")

  employee        group  age
0      Bob   Accounting   20
1     Jake  Enginerring   42
2     Lisa  Enginerring   37
3      Sue           HR   25 

   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000 



,employee,group,age,name,salary
0,Bob,Accounting,20,Bob,70000
1,Jake,Enginerring,42,Jake,80000
2,Lisa,Enginerring,37,Lisa,120000
3,Sue,HR,25,Sue,90000


In [50]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')

print(df1a, '\n\n', df2a)

# left_index, right_index : 인덱스를 이용한 병합 여부
# boolean(T/F)
pd.merge(df1a, df2a, left_index=True, right_index=True)

                group  age
employee                  
Bob        Accounting   20
Jake      Enginerring   42
Lisa      Enginerring   37
Sue                HR   25 

           hire_date  age
employee                
Lisa           2004   37
Bob            2008   20
Jake           2012   42
Sue            2014   35


,group,age_x,hire_date,age_y
employee,,,,
Bob,Accounting,20,2008,20
Jake,Enginerring,42,2012,42
Lisa,Enginerring,37,2004,37
Sue,HR,25,2014,35


In [ ]:
merge() 조인 옵션
- how=inner : 내부 조인(디폴트) => 교집합
    pd.concat() : 외부 조인이 디폴트 값
- how=outer : 외부 조인 => 합집합
- how=left : 왼쪽 조인(첫 번째 DataFrame 기준으로 조인)
- how=right : 오른쪽 조인(두 번째 DataFrame 기준으로 조인)